# LLM Optimization Modelling Experiment

In [4]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [62]:
problem = '''Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets. 

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs).'''

## 2. Generate the mathematical model

In [82]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [83]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Farmer's Crop Planning

**Parameters:**

*  $A$: Total available land (acres) = 500
*  $W_{demand}$: Wheat demand for cattle feed (tons) = 200
*  $C_{demand}$: Corn demand for cattle feed (tons) = 240
*  $P_W$: Selling price of wheat ($/ton) = 170
*  $P_C$: Selling price of corn ($/ton) = 150
*  $P_{SB}$: Selling price of sugar beets within quota ($/ton) = 36
*  $P_{SB}^E$: Selling price of sugar beets exceeding quota ($/ton) = 10
*  $Q_{SB}$: Sugar beet production quota (tons) = 6000
*  $Y_W$: Yield of wheat (tons/acre) = 2.5
*  $Y_C$: Yield of corn (tons/acre) = 3
*  $Y_{SB}$: Yield of sugar beets (tons/acre) = 20
*  $C_W$: Cost of planting wheat ($/acre) = 150
*  $C_C$: Cost of planting corn ($/acre) = 230
*  $C_{SB}$: Cost of planting sugar beets ($/acre) = 260

**Decision Variables:**

* $A_W$: Acres of land allocated to wheat
* $A_C$: Acres of land allocated to corn
* $A_{SB}$: Acres of land allocated to sugar beets
* $W_{buy}$: Tons of wheat purchased
* $C_{buy}$: Tons of corn purchased

**Objective Function:** Minimize total cost

 $$Minimize\  Z = C_W \cdot A_W + C_C \cdot A_C + C_{SB} \cdot A_{SB} + 1.4 \cdot P_W \cdot W_{buy} + 1.4 \cdot P_C \cdot C_{buy} - P_W \cdot (Y_W \cdot A_W - W_{buy}) - P_C \cdot (Y_C \cdot A_C - C_{buy}) - P_{SB} \cdot min(Y_{SB} \cdot A_{SB}, Q_{SB}) - P_{SB}^E \cdot max(0, Y_{SB} \cdot A_{SB} - Q_{SB}) $$

**Constraints:**

* **Land availability:** $A_W + A_C + A_{SB} \leq A$ 
* **Wheat demand:** $Y_W \cdot A_W + W_{buy} \geq W_{demand}$
* **Corn demand:** $Y_C \cdot A_C + C_{buy} \geq C_{demand}$
* **Non-negativity:** $A_W, A_C, A_{SB}, W_{buy}, C_{buy} \geq 0$ 

**Explanation:**

* **Objective Function:**
    * Minimizes planting costs for all crops.
    * Adds the cost of buying wheat and corn, considering the 40% markup.
    * Subtracts revenue from selling excess wheat and corn production.
    * Subtracts revenue from selling sugar beets, considering both the quota price and the excess production price.
* **Constraints:**
    * Ensures that total land allocation doesn't exceed available land.
    * Guarantees the minimum required wheat and corn for cattle feed is met through production and purchase.
    * Ensures that all decision variables are non-negative.

This model allows the farmer to determine the optimal land allocation for each crop and the amount of wheat and corn to purchase to minimize her total costs while meeting her cattle feed requirements and considering the sugar beet quota.


## 3. Generate the pyomo code

In [84]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [85]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.A = pyo.Param(initialize=500)
model.W_demand = pyo.Param(initialize=200)
model.C_demand = pyo.Param(initialize=240)
model.P_W = pyo.Param(initialize=170)
model.P_C = pyo.Param(initialize=150)
model.P_SB = pyo.Param(initialize=36)
model.P_SB_E = pyo.Param(initialize=10)
model.Q_SB = pyo.Param(initialize=6000)
model.Y_W = pyo.Param(initialize=2.5)
model.Y_C = pyo.Param(initialize=3)
model.Y_SB = pyo.Param(initialize=20)
model.C_W = pyo.Param(initialize=150)
model.C_C = pyo.Param(initialize=230)
model.C_SB = pyo.Param(initialize=260)

# Define the decision variables
model.A_W = pyo.Var(domain=pyo.NonNegativeReals)
model.A_C = pyo.Var(domain=pyo.NonNegativeReals)
model.A_SB = pyo.Var(domain=pyo.NonNegativeReals)
model.W_buy = pyo.Var(domain=pyo.NonNegativeReals)
model.C_buy = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def objective_rule(model):
    return model.C_W * model.A_W + model.C_C * model.A_C + model.C_SB * model.A_SB + 1.4 * model.P_W * model.W_buy + 1.4 * model.P_C * model.C_buy - model.P_W * (model.Y_W * model.A_W - model.W_buy) - model.P_C * (model.Y_C * model.A_C - model.C_buy) - model.P_SB * pyo.min(model.Y_SB * model.A_SB, model.Q_SB) - model.P_SB_E * pyo.max(0, model.Y_SB * model.A_SB - model.Q_SB)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define the constraints
model.land_availability = pyo.Constraint(expr=model.A_W + model.A_C + model.A_SB <= model.A)
model.wheat_demand = pyo.Constraint(expr=model.Y_W * model.A_W + model.W_buy >= model.W_demand)
model.corn_demand = pyo.Constraint(expr=model.Y_C * model.A_C + model.C_buy >= model.C_demand)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"Acres of Wheat: {pyo.value(model.A_W)}")
print(f"Acres of Corn: {pyo.value(model.A_C)}")
print(f"Acres of Sugar Beets: {pyo.value(model.A_SB)}")
print(f"Tons of Wheat to Buy: {pyo.value(model.W_buy)}")
print(f"Tons of Corn to Buy: {pyo.value(model.C_buy)}")
print(f"Total Cost: {pyo.value(model.objective)}")
```

**Sample Data:** The sample data is defined in the "Define the parameters" section of the code. For example, `model.A = pyo.Param(initialize=500)` sets the total available land to 500 acres. 


## 4. Input problem data and try running the generated code

In [86]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.A = pyo.Param(initialize=500)
model.W_demand = pyo.Param(initialize=200)
model.C_demand = pyo.Param(initialize=240)
model.P_W = pyo.Param(initialize=170)
model.P_C = pyo.Param(initialize=150)
model.P_SB = pyo.Param(initialize=36)
model.P_SB_E = pyo.Param(initialize=10)
model.Q_SB = pyo.Param(initialize=6000)
model.Y_W = pyo.Param(initialize=2.5)
model.Y_C = pyo.Param(initialize=3)
model.Y_SB = pyo.Param(initialize=20)
model.C_W = pyo.Param(initialize=150)
model.C_C = pyo.Param(initialize=230)
model.C_SB = pyo.Param(initialize=260)

# Define the decision variables
model.A_W = pyo.Var(domain=pyo.NonNegativeReals)
model.A_C = pyo.Var(domain=pyo.NonNegativeReals)
model.A_SB = pyo.Var(domain=pyo.NonNegativeReals)
model.W_buy = pyo.Var(domain=pyo.NonNegativeReals)
model.C_buy = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def objective_rule(model):
    return model.C_W * model.A_W + model.C_C * model.A_C + model.C_SB * model.A_SB + 1.4 * model.P_W * model.W_buy + 1.4 * model.P_C * model.C_buy - model.P_W * (model.Y_W * model.A_W - model.W_buy) - model.P_C * (model.Y_C * model.A_C - model.C_buy) - model.P_SB * pyo.min(model.Y_SB * model.A_SB, model.Q_SB) - model.P_SB_E * pyo.max(0, model.Y_SB * model.A_SB - model.Q_SB)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Define the constraints
model.land_availability = pyo.Constraint(expr=model.A_W + model.A_C + model.A_SB <= model.A)
model.wheat_demand = pyo.Constraint(expr=model.Y_W * model.A_W + model.W_buy >= model.W_demand)
model.corn_demand = pyo.Constraint(expr=model.Y_C * model.A_C + model.C_buy >= model.C_demand)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
print("Optimal Solution:")
print(f"Acres of Wheat: {pyo.value(model.A_W)}")
print(f"Acres of Corn: {pyo.value(model.A_C)}")
print(f"Acres of Sugar Beets: {pyo.value(model.A_SB)}")
print(f"Tons of Wheat to Buy: {pyo.value(model.W_buy)}")
print(f"Tons of Corn to Buy: {pyo.value(model.C_buy)}")
print(f"Total Cost: {pyo.value(model.objective)}")

ERROR: Rule failed when generating expression for Objective objective with
index None: AttributeError: module 'pyomo.environ' has no attribute 'min'
ERROR: Constructing component 'objective' from data=None failed:
        AttributeError: module 'pyomo.environ' has no attribute 'min'


AttributeError: module 'pyomo.environ' has no attribute 'min'

## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [88]:
print(response.text)

## Mathematical Optimization Model for Farmer's Crop Planning

**Parameters:**

*  $A$: Total available land (acres) = 500
*  $W_{demand}$: Wheat demand for cattle feed (tons) = 200
*  $C_{demand}$: Corn demand for cattle feed (tons) = 240
*  $P_W$: Selling price of wheat ($/ton) = 170
*  $P_C$: Selling price of corn ($/ton) = 150
*  $P_{SB}$: Selling price of sugar beets within quota ($/ton) = 36
*  $P_{SB}^E$: Selling price of sugar beets exceeding quota ($/ton) = 10
*  $Q_{SB}$: Sugar beet production quota (tons) = 6000
*  $Y_W$: Yield of wheat (tons/acre) = 2.5
*  $Y_C$: Yield of corn (tons/acre) = 3
*  $Y_{SB}$: Yield of sugar beets (tons/acre) = 20
*  $C_W$: Cost of planting wheat ($/acre) = 150
*  $C_C$: Cost of planting corn ($/acre) = 230
*  $C_{SB}$: Cost of planting sugar beets ($/acre) = 260

**Decision Variables:**

* $A_W$: Acres of land allocated to wheat
* $A_C$: Acres of land allocated to corn
* $A_{SB}$: Acres of land allocated to sugar beets
* $W_{buy}$: Tons of whea

In [89]:
print(response2.text)

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.A = pyo.Param(initialize=500)
model.W_demand = pyo.Param(initialize=200)
model.C_demand = pyo.Param(initialize=240)
model.P_W = pyo.Param(initialize=170)
model.P_C = pyo.Param(initialize=150)
model.P_SB = pyo.Param(initialize=36)
model.P_SB_E = pyo.Param(initialize=10)
model.Q_SB = pyo.Param(initialize=6000)
model.Y_W = pyo.Param(initialize=2.5)
model.Y_C = pyo.Param(initialize=3)
model.Y_SB = pyo.Param(initialize=20)
model.C_W = pyo.Param(initialize=150)
model.C_C = pyo.Param(initialize=230)
model.C_SB = pyo.Param(initialize=260)

# Define the decision variables
model.A_W = pyo.Var(domain=pyo.NonNegativeReals)
model.A_C = pyo.Var(domain=pyo.NonNegativeReals)
model.A_SB = pyo.Var(domain=pyo.NonNegativeReals)
model.W_buy = pyo.Var(domain=pyo.NonNegativeReals)
model.C_buy = pyo.Var(domain=pyo.NonNegativeReals)

# Define the objective function
def objective_rule(model):
   